## 1. Import Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

2025-04-10 14:56:42.020717: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2. Load Dataset Path

In [2]:
DATASET_PATH = "../data/raw"

# Image parameters
IMG_SIZE = (224, 224)  # Standard size for MobileNetV2
BATCH_SIZE = 16  # Adjust based on your hardware

### 3. Create Data Generator

In [8]:
datagen = ImageDataGenerator(
    rescale=1./255,       # Normalize pixel values
    rotation_range=20,     # Rotate images randomly
    width_shift_range=0.2, # Shift images horizontally
    height_shift_range=0.2,# Shift images vertically
    horizontal_flip=True,  # Flip images horizontally
    validation_split=0.2   # 80% train, 20% validation
)

In [9]:
# Training set
train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

NameError: name 'DATASET_PATH' is not defined

In [10]:
# Validation set
val_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

NameError: name 'DATASET_PATH' is not defined

In [6]:
# Print class labels
print("Class Mapping:", train_generator.class_indices)

Class Mapping: {'ABA_transection': 0, 'ACLIDA_transaction': 1}


### 4. Build the CNN Model

In [7]:
# Load pre-trained MobileNetV2 model without the top layer
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [8]:
# Freeze base model layers (so we don’t retrain them)
base_model.trainable = False

In [9]:
# Add custom layers for bank classification
x = Flatten()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)  # Reduce overfitting
x = Dense(len(train_generator.class_indices), activation="softmax")(x)  # Output layer

In [10]:
# Create the final model
model = Model(inputs=base_model.input, outputs=x)

In [11]:
# Compile the model
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [12]:
# Print model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 18,315,074 (69.87 MB)

 Trainable params: 16,057,090 (61.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

### 5. Train the Model

In [13]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # Adjust epochs as needed
    verbose=1
)

/Users/nychanthrith/Documents/Chantharith/I3-S2/Programming for DS/TP/Project/Bank-Transaction-Scanner/venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.7396 - loss: 7.1689 - val_accuracy: 0.9153 - val_loss: 2.0672
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9086 - loss: 3.7061 - val_accuracy: 0.9831 - val_loss: 0.0219
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9405 - loss: 1.4058 - val_accuracy: 0.9661 - val_loss: 0.4849
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9229 - loss: 2.1423 - val_accuracy: 1.0000 - val_loss: 4.8470e-05
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.9651 - loss: 1.1961 - val_accuracy: 0.9831 - val_loss: 0.2519
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.9595 - loss: 1.0871 - val_accuracy: 0.9492 - val_loss: 1.3913
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.9771 - loss: 0.5538 - val_accuracy: 0.9492 - val_loss: 1.4288
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9605 - loss: 0.7887 - val_accuracy: 0.9322 - val_l

In [14]:
model.save("../models/bank_classification.h5")


## Test the Model on a New Image

### Import Libraries

In [2]:
import numpy as np
from tensorflow.keras.preprocessing import image

### Function to predict bank from an image

In [3]:
def predict_bank(image_path, model, class_indices):
    img = image.load_img(image_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make prediction
    predictions = model.predict(img_array)
    class_id = np.argmax(predictions)

    # Get class label
    labels = {v: k for k, v in class_indices.items()}
    return labels[class_id]

### Load trained model

In [5]:
file = "../src/backend/models/bank_classification.h5"
model = tf.keras.models.load_model(file)

### Test with a new image

In [6]:
test_image = "../data/raw/ABA_test.PNG"
predicted_bank = predict_bank(test_image, model, train_generator.class_indices)
print("Predicted Bank:", predicted_bank)

NameError: name 'train_generator' is not defined

In [ ]:
test_image = "/content/photo_2025-03-24 23.55.00.jpeg"
predicted_bank = predict_bank(test_image, model, train_generator.class_indices)
print("Predicted Bank:", predicted_bank)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted Bank: ABA_transection


In [20]:
# prompt: it looks like our trained data is good with ABA_transaction but not ACLIDA_transaction. When I choose a picture from ACLIDA_transaction then it shows it is ABA_transaction

import numpy as np
# Find the number of images in each class
import os

def count_images_per_class(dataset_path):
    class_counts = {}
    for class_name in os.listdir(dataset_path):
        class_dir = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_dir):
            image_count = len([f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))])
            class_counts[class_name] = image_count
    return class_counts


counts = count_images_per_class(DATASET_PATH)
print(counts)

# Analyze the class distribution.  If one class has significantly fewer examples,
# the model may be biased towards the more numerous class.
# Consider augmenting the data for the underrepresented class, or collecting more
# data for that class.

# Examine misclassified images:
# 1.  Review the images that are misclassified.  Are there any common characteristics
#     among the misclassified ACLIDA images?  For example, are the images poorly lit,
#     blurry, or at unusual angles?
# 2.  Look at the predicted probabilities:  The model might have some confidence in its
#     incorrect prediction.  If the model is only slightly more confident in the wrong class,
#     that suggests the images might be difficult to distinguish.  Review the prediction probabilities.


# Data Augmentation (if class imbalance is significant):
# Increase the number of ACLIDA images through data augmentation.  Consider more aggressive augmentations like:
# - wider rotation range
# - different zoom ranges
# - more shear range

# Adjust the model:
# - Try a different model architecture.
# - Add more layers, change the number of neurons in dense layers.
# - Fine-tune more layers of the base model (MobileNetV2).  Start by unfreezing a few top layers,
#   and gradually unfreeze more layers as you train.  Monitor the validation accuracy carefully
#   to avoid overfitting.
# - Experiment with different optimizers and learning rates.


# Collect more data:
# The most reliable solution is often to collect more representative images of ACLIDA_transaction.

#Example of how to print prediction probability
def predict_bank_with_probability(image_path, model, class_indices):
    img = image.load_img(image_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make prediction
    predictions = model.predict(img_array)
    class_id = np.argmax(predictions)

    # Get class label and probability
    labels = {v: k for k, v in class_indices.items()}
    predicted_label = labels[class_id]
    predicted_probability = predictions[0][class_id]

    return predicted_label, predicted_probability

# Example usage:
test_image = "/content/photo_2025-03-24 23.55.00.jpeg"
predicted_bank, probability = predict_bank_with_probability(test_image, model, train_generator.class_indices)
print("Predicted Bank:", predicted_bank)
print(f"Probability: {probability}")


{'ABA_transection': 210, 'ACLIDA_transaction': 93}


FileNotFoundError: [Errno 2] No such file or directory: '/content/photo_2025-03-24 23.55.00.jpeg'

# Pattern Model for Each Bank

In [ ]:
bank_name = predict_bank(image_path)

NameError: name 'predict_bank' is not defined